##  <font color = "navy">WEATHER FORECAST IN AUSTRALIA: Modélisation deep learning</font>
Notebook <b>3/3</b>
***

<font color = "purple"><b>Objectif: Réaliser un modèle de deep learning capable de détecter les jours de pluie à J+1 avec une précision satisfaisante.</b></font>

<b> Résumé du notebook : </b>
- Data cleaning, preprocessing des données (notamment encodage, et rééquilibrage) ;
- Modélisation deep learning ;
- Essai d'amélioration des performances du modèle.

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import warnings
from sklearn.preprocessing import LabelEncoder
from IPython.display import Markdown, display
warnings.filterwarnings('ignore')
sns.set_theme({'legend.frameon':True})
from tensorflow.keras.layers import Input, Dense 
from tensorflow.keras.models import Model
import itertools 
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import ensemble
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

#pip install --upgrade tensorflow
#pip install pydot
#pip install graphviz

In [19]:
#Import d'une version prête à l'emploi (preprocessing: Cf notebook 2/4)
data = pd.read_csv("weatherAUS_DL.csv")

#Création d'un df de secours pour avoir une trace des données brutes.

df_saved = pd.read_csv("weatherAUS_DL.csv")

pd.set_option("display.max_columns", None)

feats = data.drop(["RainTomorrow", "Date", "Year", "Month", "Day", "State", "Location"], axis = 1)
target = data["RainTomorrow"]
encoder = LabelEncoder()

Y = encoder.fit_transform(target)

X_train, X_test, y_train, y_test = train_test_split(feats, Y, test_size = 0.2, random_state = 55) 

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
display(Markdown("""<h2><u><center>Réalisation d'un premier modèle de deep learning</center></u></h2>
<h4><u>Entrainement du modèle :</u></h4>"""))
EPOCHS = 6
BATCHS = 32

UNITS1 = 25
UNITS2 = 50
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None

inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")

x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)
print(score)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce premier modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</u></h4>
<ul><li>Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("""Les paramètres de ce premier modèle ont été fixés de façon plus ou moins arbritraire. Les 
performances de ce modèle sont globalement satisfaisantes, voire très satisfaisantes pour la détection des jours sans pluie.
Il serait toutefois intéressant d'améliorer la prédiction de jours de pluie."""))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><u><center>Réalisation d'un premier modèle de deep learning</center></u></h2>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
3229/3229 [==============================] - 3s 912us/step - loss: 0.3541 - accuracy: 0.8460 - val_loss: 0.3372 - val_accuracy: 0.8548
Epoch 2/6
3229/3229 [==============================] - 3s 875us/step - loss: 0.3440 - accuracy: 0.8510 - val_loss: 0.3368 - val_accuracy: 0.8543
Epoch 3/6
3229/3229 [==============================] - 3s 875us/step - loss: 0.3420 - accuracy: 0.8515 - val_loss: 0.3386 - val_accuracy: 0.8539
Epoch 4/6
3229/3229 [==============================] - 3s 872us/step - loss: 0.3402 - accuracy: 0.8529 - val_loss: 0.3356 - val_accuracy: 0.8550
Epoch 5/6
3229/3229 [==============================] - 3s 866us/step - loss: 0.3386 - accuracy: 0.8534 - val_loss: 0.3335 - val_accuracy: 0.8555
Epoch 6/6
808/808 [==============================] - 0s 587us/step - loss: 0.3324 - accuracy: 0.8561
[0.33237820863723755, 0.8560752868652344]


<h4><u>Récapitulatif du modèle :</u></h4>
Ce premier modèle avait 2 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</u></h4>
<ul><li>Prédiction des <b>jours de pluie</b> avec une précision de <b>75.47 %</b> (<i>recall = <b>49.39 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>87.28 %</b> (<i>recall = <b>95.58 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.61 %</b>.</li></ul>


Les paramètres de ce premier modèle ont été fixés de façon plus ou moins arbritraire. Les 
performances de ce modèle sont globalement satisfaisantes, voire très satisfaisantes pour la détection des jours sans pluie.
Il serait toutefois intéressant d'améliorer la prédiction de jours de pluie.

<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.87      0.96      0.91     20250
           1       0.75      0.49      0.60      5576

    accuracy                           0.86     25826
   macro avg       0.81      0.72      0.75     25826
weighted avg       0.85      0.86      0.84     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19355   895
1                2822  2754


In [22]:
display(Markdown("""<h2><center><u>Effet du nombre de neurones</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 6
BATCHS = 32

UNITS1 = 250
UNITS2 = 500
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None

inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")


x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))
display(Markdown("""Le simple ajout de neurones (ici 10 fois plus nombreux dans chaque couche par rapport au modèle initial) 
ne semble pas modifier les performances."""))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><center><u>Effet du nombre de neurones</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3495 - accuracy: 0.8492 - val_loss: 0.3411 - val_accuracy: 0.8542
Epoch 2/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3408 - accuracy: 0.8525 - val_loss: 0.3321 - val_accuracy: 0.8563
Epoch 3/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3369 - accuracy: 0.8542 - val_loss: 0.3304 - val_accuracy: 0.8574
Epoch 4/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3338 - accuracy: 0.8557 - val_loss: 0.3331 - val_accuracy: 0.8568
Epoch 5/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3318 - accuracy: 0.8566 - val_loss: 0.3346 - val_accuracy: 0.8577
Epoch 6/6
808/808 [==============================] - 1s 764us/step - loss: 0.3325 - accuracy: 0.8567


<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>250 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>500 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 250)               5750      
                                                                 
 Dense_2 (Dense)             (None, 500)               125500    
                                                                 
 Dense_3 (Dense)             (None, 2)                 1002      
                                                                 
Total params: 132,252
Trainable params: 132,252
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>75.15 %</b> (<i>recall = <b>50.22 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>87.44 %</b> (<i>recall = <b>95.43 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.67 %</b>.</li></ul>


Le simple ajout de neurones (ici 10 fois plus nombreux dans chaque couche par rapport au modèle initial) 
ne semble pas modifier les performances.

<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.87      0.95      0.91     20250
           1       0.75      0.50      0.60      5576

    accuracy                           0.86     25826
   macro avg       0.81      0.73      0.76     25826
weighted avg       0.85      0.86      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19324   926
1                2776  2800


In [23]:
display(Markdown("""<h2><center><u>Effet du nombre d'épochs</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 18
BATCHS = 32

UNITS1 = 25
UNITS2 = 50
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None

inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")

x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))
display(Markdown("""L'entrainement sur un nombre plus important d'epochs (ici 18 soit 3 fois plus par rapport au modèle initial) 
ne semble pas modifier les performances."""))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><center><u>Effet du nombre d'épochs</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/18
3229/3229 [==============================] - 3s 977us/step - loss: 0.3545 - accuracy: 0.8464 - val_loss: 0.3410 - val_accuracy: 0.8528
Epoch 2/18
3229/3229 [==============================] - 3s 858us/step - loss: 0.3434 - accuracy: 0.8509 - val_loss: 0.3393 - val_accuracy: 0.8526
Epoch 3/18
3229/3229 [==============================] - 3s 862us/step - loss: 0.3416 - accuracy: 0.8519 - val_loss: 0.3350 - val_accuracy: 0.8553
Epoch 4/18
3229/3229 [==============================] - 3s 867us/step - loss: 0.3397 - accuracy: 0.8525 - val_loss: 0.3337 - val_accuracy: 0.8559
Epoch 5/18
3229/3229 [==============================] - 3s 864us/step - loss: 0.3388 - accuracy: 0.8523 - val_loss: 0.3329 - val_accuracy: 0.8551
Epoch 6/18
3229/3229 [==============================] - 3s 867us/step - loss: 0.3373 - accuracy: 0.8533 - val_loss: 0.3326 - val_accuracy: 0.8558
Epoch 7/18
3229/3229 [==============================] - 3s 873us/step - loss: 0.3367 - accuracy: 0.8535 - val_loss: 0.3321 -

<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>18 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>74.05 %</b> (<i>recall = <b>52.24 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>87.84 %</b> (<i>recall = <b>94.96 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.74 %</b>.</li></ul>


L'entrainement sur un nombre plus important d'epochs (ici 18 soit 3 fois plus par rapport au modèle initial) 
ne semble pas modifier les performances.

<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.95      0.91     20250
           1       0.74      0.52      0.61      5576

    accuracy                           0.86     25826
   macro avg       0.81      0.74      0.76     25826
weighted avg       0.85      0.86      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19229  1021
1                2663  2913


In [24]:
display(Markdown("""<h2><center><u>Effet de la taille des batchs</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 6
BATCHS = 320

UNITS1 = 25
UNITS2 = 50
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None

inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")

x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)

model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()
                 
display(Markdown("""<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("""L'entrainement sur des batchs de plus grande taille (ici 320 soit 10 fois plus grands par rapport au modèle
initial) ne semble pas modifier les performances, en revanche la vitesse d'execution du modèle est considérablement réduite.
"""))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><center><u>Effet de la taille des batchs</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
323/323 [==============================] - 1s 2ms/step - loss: 0.3850 - accuracy: 0.8315 - val_loss: 0.3430 - val_accuracy: 0.8528
Epoch 2/6
323/323 [==============================] - 0s 1ms/step - loss: 0.3468 - accuracy: 0.8500 - val_loss: 0.3403 - val_accuracy: 0.8536
Epoch 3/6
323/323 [==============================] - 0s 1ms/step - loss: 0.3449 - accuracy: 0.8503 - val_loss: 0.3388 - val_accuracy: 0.8534
Epoch 4/6
323/323 [==============================] - 0s 1ms/step - loss: 0.3439 - accuracy: 0.8507 - val_loss: 0.3400 - val_accuracy: 0.8536
Epoch 5/6
323/323 [==============================] - 0s 1ms/step - loss: 0.3426 - accuracy: 0.8510 - val_loss: 0.3378 - val_accuracy: 0.8558
Epoch 6/6
808/808 [==============================] - 1s 610us/step - loss: 0.3383 - accuracy: 0.8542


<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>320</b>.</li></ul>


Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>72.24 %</b> (<i>recall = <b>52.73 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>87.88 %</b> (<i>recall = <b>94.42 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.42 %</b>.</li></ul>


L'entrainement sur des batchs de plus grande taille (ici 320 soit 10 fois plus grands par rapport au modèle
initial) ne semble pas modifier les performances, en revanche la vitesse d'execution du modèle est considérablement réduite.


<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.94      0.91     20250
           1       0.72      0.53      0.61      5576

    accuracy                           0.85     25826
   macro avg       0.80      0.74      0.76     25826
weighted avg       0.85      0.85      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19120  1130
1                2636  2940


In [25]:
display(Markdown("""<h2><center><u>Effet des fonctions d'activation</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 6
BATCHS = 32

UNITS1 = 25
UNITS2 = 50
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "tanh"
ACTIV2 = "tanh"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None


inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")

x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("""L'activation des couches denses par la fonction tangente hyperbolique (au lieu de relu dans modèle initial) 
ne semble pas modifier les performances."""))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><center><u>Effet des fonctions d'activation</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
3229/3229 [==============================] - 3s 909us/step - loss: 0.3593 - accuracy: 0.8444 - val_loss: 0.3494 - val_accuracy: 0.8514
Epoch 2/6
3229/3229 [==============================] - 3s 875us/step - loss: 0.3540 - accuracy: 0.8463 - val_loss: 0.3460 - val_accuracy: 0.8508
Epoch 3/6
3229/3229 [==============================] - 3s 898us/step - loss: 0.3508 - accuracy: 0.8483 - val_loss: 0.3439 - val_accuracy: 0.8501
Epoch 4/6
3229/3229 [==============================] - 3s 869us/step - loss: 0.3481 - accuracy: 0.8486 - val_loss: 0.3415 - val_accuracy: 0.8537
Epoch 5/6
3229/3229 [==============================] - 3s 876us/step - loss: 0.3469 - accuracy: 0.8485 - val_loss: 0.3419 - val_accuracy: 0.8532
Epoch 6/6
808/808 [==============================] - 0s 589us/step - loss: 0.3410 - accuracy: 0.8533


<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>tanh</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>tanh</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>71.67 %</b> (<i>recall = <b>52.99 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>87.92 %</b> (<i>recall = <b>94.23 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.33 %</b>.</li></ul>


L'activation des couches denses par la fonction tangente hyperbolique (au lieu de relu dans modèle initial) 
ne semble pas modifier les performances.

<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.94      0.91     20250
           1       0.72      0.53      0.61      5576

    accuracy                           0.85     25826
   macro avg       0.80      0.74      0.76     25826
weighted avg       0.84      0.85      0.84     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19082  1168
1                2621  2955


In [26]:
display(Markdown("""<h2><center><u>Effet du nombre de couches de neurones</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 6
BATCHS = 32

UNITS1 = 25
UNITS2 = 50
UNITS3 = 50
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = "relu"
ACTIV4 = None
ACTIV5 = None


inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITS3, activation = ACTIV3, kernel_initializer = "normal", name = "Dense_3")
dense4 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_4")

x = dense1(inputs)
x = dense2(x)
x = dense3(x)
outputs = dense4(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 3 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la troisième avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, UNITS3, ACTIV3, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("""L'ajout d'une troisième couche de neurones ne semble pas modifier les performances.
"""))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><center><u>Effet du nombre de couches de neurones</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
3229/3229 [==============================] - 3s 979us/step - loss: 0.3541 - accuracy: 0.8467 - val_loss: 0.3398 - val_accuracy: 0.8536
Epoch 2/6
3229/3229 [==============================] - 3s 916us/step - loss: 0.3449 - accuracy: 0.8508 - val_loss: 0.3372 - val_accuracy: 0.8542
Epoch 3/6
3229/3229 [==============================] - 3s 918us/step - loss: 0.3425 - accuracy: 0.8513 - val_loss: 0.3397 - val_accuracy: 0.8548
Epoch 4/6
3229/3229 [==============================] - 3s 921us/step - loss: 0.3406 - accuracy: 0.8522 - val_loss: 0.3342 - val_accuracy: 0.8553
Epoch 5/6
3229/3229 [==============================] - 3s 918us/step - loss: 0.3392 - accuracy: 0.8527 - val_loss: 0.3335 - val_accuracy: 0.8552
Epoch 6/6
808/808 [==============================] - 1s 749us/step - loss: 0.3333 - accuracy: 0.8541


<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 3 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la troisième avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 50)                2550      
                                                                 
 Dense_4 (Dense)             (None, 2)                 102       
                                                                 
Total params: 4,527
Trainable params: 4,527
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>74.11 %</b> (<i>recall = <b>49.86 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>87.33 %</b> (<i>recall = <b>95.2 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.41 %</b>.</li></ul>


L'ajout d'une troisième couche de neurones ne semble pas modifier les performances.


<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.87      0.95      0.91     20250
           1       0.74      0.50      0.60      5576

    accuracy                           0.85     25826
   macro avg       0.81      0.73      0.75     25826
weighted avg       0.84      0.85      0.84     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19279   971
1                2796  2780
